In [2]:
import pandas as pd
import glob
import os
import numpy as np


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
df_2019_food = pd.read_csv('data/sampled/food_2019.csv')
df_2020_food = pd.read_csv('data/sampled/food_2020.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [205]:
df_sub_pulled = pd.read_csv('data/sampled/subs_relevant.csv')
df_com_pulled = pd.read_csv('data/sampled/coms_relevant.csv')

In [206]:
submission_columns = ['author', 'subreddit', 'title', 'selftext', 'score', 'url', 'created_utc', 'full_link']
comment_columns = ['author', 'subreddit', 'body', 'score', 'created_utc', 'permalink']

In [208]:
# all submissions
df_food = pd.concat([df_2019_food, df_2020_food])
df_allsubs = pd.concat([df_sub_pulled, df_food])
df_allsubs = df_allsubs.loc[df_allsubs.author != '[deleted]', submission_columns + ['removed_by_category']]
df_allsubs['date'] = pd.to_datetime(df_allsubs.created_utc, unit='s')
df_allsubs['type'] = 'submission'
df_allsubs.sort_values(['author', 'date'], ascending=[True, True], inplace=True)
df_allsubs.rename({'selftext': 'body', 'url': 'img_url', 'full_link': 'link', 'score': 'reddit_score'}, axis=1, inplace=True)
df_allsubs.head()

,author,subreddit,title,body,reddit_score,img_url,created_utc,link,removed_by_category,date,type
12458,------me,food,[homemade] spaghetti &amp; meatballs,NaN,1,https://i.redd.it/h4871wtlmoq41.jpg,1585954214,https://www.reddit.com/r/food/comments/fuiccv/...,NaN,2020-04-03 22:50:14,submission
5391,------me,food,[Homemade] Chicken Tikka Masala &amp; Saag,NaN,21,https://i.redd.it/6c579t8y6hs41.jpg,1586735895,https://www.reddit.com/r/food/comments/g07oo2/...,NaN,2020-04-12 23:58:15,submission
822,------me,food,[Homemade] Porcini-crusted filet mignon,NaN,47,https://i.redd.it/pxyzmita5ot41.jpg,1587255935,https://www.reddit.com/r/food/comments/g3y7zo/...,NaN,2020-04-19 00:25:35,submission
4736,---Alexander---,food,"[Homemade] Cauliflower on mashed potatoes, cov...",NaN,5,https://i.redd.it/0y1ye7vgmpq21.jpg,1554586019,https://www.reddit.com/r/food/comments/ba95ic/...,NaN,2019-04-06 21:26:59,submission
10187,---ShineyHiney---,food,"[Homemade] “Quarantine Quiche:” Bacon, Cheese,...",NaN,3,https://i.redd.it/wkk2ceku49r41.jpg,1586202489,https://www.reddit.com/r/food/comments/fw5kvh/...,moderator,2020-04-06 19:48:09,submission


In [209]:
df_com_pulled = df_com_pulled.loc[df_com_pulled.author != '[deleted]', :][comment_columns]
df_com_pulled['date'] = pd.to_datetime(df_com_pulled.created_utc, unit='s')
df_com_pulled['type'] = 'comment'
df_com_pulled['permalink'] = df_com_pulled.permalink.apply(lambda t: 'https://www.reddit.com' + t)
df_com_pulled.rename({'permalink': 'link', 'score': 'reddit_score'}, axis=1, inplace=True)
df_com_pulled

,author,subreddit,body,reddit_score,created_utc,link,date,type
0,I_am_Bob,food,It depends on where you are on the east coast....,2,1536075341,https://www.reddit.com/r/food/comments/9cnwcc/...,2018-09-04 15:35:41,comment
1,eddiewastaken,food,"No offence Lynn, but your life is *technically...",2,1535669515,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 22:51:55,comment
2,eddiewastaken,food,Black pudding mate.,2,1535659295,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 20:01:35,comment
3,eddiewastaken,food,But then how do I know there’s enough fried fo...,1,1535658957,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 19:55:57,comment
4,eddiewastaken,food,Try nicking a sausage from an Englishman’s fry...,7,1535655774,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 19:02:54,comment
...,...,...,...,...,...,...,...,...
439602,gumball_Jones,food,Best Sunday morning food ever. Living in Barce...,-24,1636276842,https://www.reddit.com/r/food/comments/qokzmk/...,2021-11-07 09:20:42,comment
439603,gumball_Jones,GetMotivated,About to take a nap. I feel this was written f...,2,1635513250,https://www.reddit.com/r/GetMotivated/comments...,2021-10-29 13:14:10,comment
439604,kfletch16,offmychest,They can hire some and Payless. Plus they are ...,3,1638233251,https://www.reddit.com/r/offmychest/comments/r...,2021-11-30 00:47:31,comment
439605,gumball_Jones,food,Here in Spain it’s not.,-1,1638106650,https://www.reddit.com/r/food/comments/r4197j/...,2021-11-28 13:37:30,comment


### Sentiment

In [256]:
from sentiment import score_submissions_sentistrength, score_submissions_vader, score_comments_vader, score_comments_sentistrength

In [265]:
df_allsubs = score_submissions_vader(df_allsubs, col_selftext='body')
df_allsubs.body_s_vader = df_allsubs.body_s_vader.fillna(0)
df_allsubs = score_submissions_sentistrength(df_allsubs, col_selftext='body')
df_allsubs.title_s_vader = df_allsubs.body_s_vader.fillna(0)

In [271]:
df_com_pulled = score_comments_vader(df_com_pulled, col_body='body')
df_com_pulled = score_comments_sentistrength(df_com_pulled, col_body='body')

In [278]:
df_com_pulled.head()

,author,subreddit,body,reddit_score,created_utc,link,date,type,body_s_vader,body_s_sst
0,I_am_Bob,food,It depends on where you are on the east coast....,2,1536075341,https://www.reddit.com/r/food/comments/9cnwcc/...,2018-09-04 15:35:41,comment,0.6808,1
1,eddiewastaken,food,"No offence Lynn, but your life is *technically...",2,1535669515,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 22:51:55,comment,-0.1419,-1
2,eddiewastaken,food,Black pudding mate.,2,1535659295,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 20:01:35,comment,0.0000,1
3,eddiewastaken,food,But then how do I know there’s enough fried fo...,1,1535658957,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 19:55:57,comment,0.0000,1
4,eddiewastaken,food,Try nicking a sausage from an Englishman’s fry...,7,1535655774,https://www.reddit.com/r/food/comments/9blpez/...,2018-08-30 19:02:54,comment,0.4062,0


### Concat

In [279]:
df_reddit = pd.concat([df_com_pulled, df_allsubs])
df_reddit = df_reddit.loc[(df_reddit.title != '[deleted]') & (df_reddit.body != '[deleted]') & pd.isna(df_reddit.removed_by_category), :]
df_reddit = df_reddit[['author', 'date', 'subreddit', 'type', 'title', 'body', 'img_url', 'title_s_vader', 'title_s_sst', 'body_s_vader', 'body_s_sst', 'link']].sort_values(['author', 'date', 'subreddit', 'type']).drop_duplicates()
df_reddit.head()

,author,date,subreddit,type,title,body,img_url,title_s_vader,title_s_sst,body_s_vader,body_s_sst,link
268997,------me,2019-09-06 02:07:56,food,comment,NaN,This is the best sub at Firehouse. Used to liv...,NaN,NaN,NaN,0.5574,0,https://www.reddit.com/r/food/comments/d05ouu/...
269166,------me,2019-10-06 23:11:32,food,comment,NaN,"Burger sauce: Mayo, bbq sauce, ketchup, relish...",NaN,NaN,NaN,0.0000,0,https://www.reddit.com/r/food/comments/de9emu/...
269112,------me,2019-11-03 14:10:27,food,comment,NaN,Wish I could order this somewhere near me.,NaN,NaN,NaN,0.4019,0,https://www.reddit.com/r/food/comments/dqxhta/...
269242,------me,2019-11-28 01:01:38,food,comment,NaN,"Aww, thank you!",NaN,NaN,NaN,0.4199,2,https://www.reddit.com/r/food/comments/e2pi63/...
269237,------me,2019-12-08 15:29:49,food,comment,NaN,This looks amazing.,NaN,NaN,NaN,0.5859,2,https://www.reddit.com/r/food/comments/e7t4zj/...


# Now we have: 
- people who submitted to food
- people who submitted to other relevant
- people that commented anywhere

 get people that submitted to food and actually posted somewhere else

In [221]:
df_2019_relevant = df_reddit[(df_reddit.subreddit != 'food') & (df_reddit.author.isin(df_2019_food.author)) & (df_reddit.author != 'AutoModerator')]
df_2020_relevant = df_reddit[(df_reddit.subreddit != 'food') & (df_reddit.author.isin(df_2020_food.author)) & (df_reddit.author != 'AutoModerator')]
authors_2019_relevant = df_2019_relevant.author.unique()
authors_2020_relevant = df_2020_relevant.author.unique()

In [222]:
print('Food submitters in march-april 2019:', len(df_2019_food.author.unique()), 'Of those, how many commented (or posted) in other subreddits than food?', len(authors_2019_relevant))
print('Food submitters in march-april 2020:', len(df_2020_food.author.unique()), 'Of those, how many commented (or posted) in other subreddits than food?', len(authors_2020_relevant))

Food submitters in march-april 2019: 7350 Of those, how many commented (or posted) in other subreddits than food? 1635
Food submitters in march-april 2020: 11838 Of those, how many commented (or posted) in other subreddits than food? 2263


In [227]:
author_sample_2019 = np.random.choice(df_2019_relevant.author, size=50, replace=False)
author_sample_2020 = np.random.choice(list(set(df_2020_relevant.author) - set(author_sample_2019)), size=50, replace=False)

In [245]:
def show_author_posts(df, author):
    return df[df.author == author].sort_values(['date', 'type'])


def save_author_posts_separate(df, sampled_authors, prefix):
    for author in sampled_authors:
        df_author = show_author_posts(df, author).copy()
        df_author.to_csv(f'data/sampled/authors_separate/{prefix}_{author}.csv', index=False, sep=';')

def save_author_posts_collected(df, sampled_authors, prefix):
    df[df.author.isin(sampled_authors)].sort_values(['author', 'date', 'subreddit', 'type']).to_csv(f'data/sampled/authors_collected/{prefix}_ALL.csv', index=False, sep=';')

In [284]:
save_author_posts_separate(df_reddit, author_sample_2019, 'sample2019')
save_author_posts_separate(df_reddit, author_sample_2020, 'sample2020')

In [281]:
save_author_posts_collected(df_reddit, author_sample_2019, 'sample2019')
save_author_posts_collected(df_reddit, author_sample_2020, 'sample2020')

In [282]:
# see other notebook about calories
df_reddit[df_reddit.author.isin((set(author_sample_2019) | set(author_sample_2020))) & (df_reddit.subreddit == 'food') & (df_reddit.type == 'submission')].to_csv('data/sampled/author_food_sample/sampled_food.csv', index=False)

In [286]:
df_author = show_author_posts(df_reddit, 'kiyannepeppers')
df_author[df_author.type == 'submission']

,author,date,subreddit,type,title,body,img_url,title_s_vader,title_s_sst,body_s_vader,body_s_sst,link
46838,kiyannepeppers,2018-12-14 03:58:16,depression,submission,Tired,Why am I so depressed when I have no reason to...,https://www.reddit.com/r/depression/comments/a...,-0.9597,-1.0,-0.9597,-1,https://www.reddit.com/r/depression/comments/a...
46837,kiyannepeppers,2019-01-01 17:02:29,Anxiety,submission,Dors anyone else develop an anxiety from dreams?,I ask because im usually always having bad dre...,https://www.reddit.com/r/Anxiety/comments/abjp...,-0.8825,-2.0,-0.8825,-3,https://www.reddit.com/r/Anxiety/comments/abjp...
46836,kiyannepeppers,2019-01-01 17:13:02,Anxiety,submission,Has anyone else developed a new anxiety from b...,I ask because im usually always having bad dre...,https://www.reddit.com/r/Anxiety/comments/abjt...,-0.9429,-2.0,-0.9429,-2,https://www.reddit.com/r/Anxiety/comments/abjt...
9992,kiyannepeppers,2019-03-24 02:18:53,food,submission,[Homemade] teriyaki meatballs with lots of veg...,NaN,https://i.redd.it/h1atfmh68zn21.jpg,0.0000,1.0,0.0000,0,https://www.reddit.com/r/food/comments/b4raia/...
46839,kiyannepeppers,2019-05-22 16:36:45,Anxiety,submission,Im terrified of driving,"So back story, I'm from Canada (BC) and our sy...",https://www.reddit.com/r/Anxiety/comments/brqy...,-0.9942,-3.0,-0.9942,-1,https://www.reddit.com/r/Anxiety/comments/brqy...
46843,kiyannepeppers,2020-02-07 06:51:54,depression,submission,Does adulthood just suck forever?,I thought my depression was bad in highschool....,https://www.reddit.com/r/depression/comments/f...,-0.9694,-1.0,-0.9694,-1,https://www.reddit.com/r/depression/comments/f...
46859,kiyannepeppers,2021-08-25 04:58:42,depression,submission,I'm struggling to find a reason to live,"Like what is the point? I wake up, do nothing ...",https://www.reddit.com/r/depression/comments/p...,-0.9481,-2.0,-0.9481,-1,https://www.reddit.com/r/depression/comments/p...
46860,kiyannepeppers,2022-01-02 07:33:49,Anxiety,submission,Getting sick of this,I'd love to just go ONE day without feeling an...,https://www.reddit.com/r/Anxiety/comments/ru4l...,-0.9865,-1.0,-0.9865,-1,https://www.reddit.com/r/Anxiety/comments/ru4l...
